In [8]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import pandas as pd
from langgraph.graph import StateGraph,START,END
from langchain_core.messages import HumanMessage
from pydantic import BaseModel
from typing import List

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini")


In [12]:
movie_data_set = pd.read_csv('./sample_100mb.csv')

In [ ]:
class DataState(BaseModel):
    user_query : str
    columns : List[str]
    dataset_info : str
    dataset_name : str
    response : str
    error_message : str


# df.head().to_string(index=False)

In [ ]:
def process_user_query(state:DataState):
    return f""" You are an AI assistant for data analysis. The dataset has following columns : {state['columns']}.
    And data looks like this : {state['dataset_info']}.And the name of the dataset is : {state['dataset_name']}
    Given the user query : {state['user_query']}, generate the python code to execute the query on the dataset.
    """
def generate_error_prompt(state:DataState):
    return f"""
    The generated code caused the following error: {state['error_message']}.
    Please rectify the error and regenerate the correct Python code for the query: "{state['user_query']}".
    """
    
def generate_llm_response(state : DataState):
    response = llm.invoke({"role":"user","query":HumanMessage(content=process_user_query(state=state))})
    print("response")
    return {state['response']:response}
    

In [ ]:
def fail_condition(state:DataState):
    pass

def execute_query(state:DataState):
    local_vars = {"movie_df": movie_data_set}
    try: 
        exec(state['response'])
    except Exception as e:
        print("LLm Response :",state['response'] ,"\n\n")
        print("Error : ", e)